# **Usage Agent**

The **Usage Agent's** functionnality is to estimate the probability that a particular device will be used on the following day. Within the general recommendation framework, this function is used in order to limit unnecessary recommendations that could irritate the user. Whenever a device is unlikely to be used on the next day (estimated likelihood below a certain threshold), no recommendation will be made.

In the present notebook, we will describe how these probabilities are estimated in detail and define the **Usage Agent** class that will be integrated into the recommendation agent.

The Usage Agent will use a ML-algorithm on features extracted from the household's electricity consumption data in order to predict the likelihood of use of devices on the next day. For instance, at a given day t-1, it will use all available consumption data until day t-1 in order to predict device usage on day t. The features we will use can be divided into 3 categories: 
1. Whether activity has been detected in the house in the preceding days (activity detected by electricity consumption).
2. Whether the to-be-prediced-device has been used in the previous days.
3. Time dummies.

Given the limited number of observations for each household, we will need to restrict the complexity of the ML-Algorithm in use. This is the reason why we will use a logit model with a limited number of features.

## **1. Load And Preprocess Data**

This part's only purpose is to load the data used in the Usage Agent. This process is described in detail in the Preparation Agent. 

**Note: When computing the script with another Household than Household 1 you might need to adapt some parameters**

### **1.1 Initialize And Load Python Scripts**

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
dir = 'D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main'
os.chdir(dir)

from helper_functions import Helper
from agents import Preparation_Agent
import pandas as pd

helper = Helper()

dbfile  = "D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main/home-assistant_Chris_v3.db"


C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
import numpy as np
import pandas
import sklearn
import statsmodels
from interpret.glassbox.ebm.ebm import ExplainableBoostingClassifier     
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost
import statsmodels.api
import datetime

### **1.2 Set Parameters For Pre-processing Step**

In [4]:
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]

truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}
aggregate_params24_H = {
    'resample_param': '24H'
}


activity_params = {
    'active_appliances': shiftable_devices,
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}



device = {
    'threshold' : .15}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}

date = '2023-01-08'
model_type = 'random forest'

### **1.3 Pre-process Data For Input In Device_Usage Agent**

In [5]:
# calling the preparation pipeline
prep = Preparation_Agent(dbfile, shiftable_devices)
df = prep.pipeline_usage(prep.input, usage_pipe_params)

#display all potential variables for predicting device usage likelihood
df

,activity,sensor.shellyplug_s_4022d88961b4_power_usage,sensor.shellyplug_s_4022d88984b8_power_usage,periods_since_last_activity,periods_since_last_sensor.shellyplug_s_4022d88961b4_power_usage,periods_since_last_sensor.shellyplug_s_4022d88984b8_power_usage,hour,activity_lag_1,activity_lag_2,activity_lag_3,...,sensor.shellyplug_s_4022d88984b8_power_usage_lag_1,sensor.shellyplug_s_4022d88984b8_power_usage_lag_2,sensor.shellyplug_s_4022d88984b8_power_usage_lag_3,active_last_2_days,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
last_updated,,,,,,,,,,,,,,,,,,,,,
2022-12-25,1,1,1,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,1,0,0,0
2022-12-26,1,1,1,1.0,1.0,1.0,0,1.0,NaN,NaN,...,1.0,NaN,NaN,1,1,0,0,0,0,0
2022-12-27,1,1,1,1.0,1.0,1.0,0,1.0,1.0,NaN,...,1.0,1.0,NaN,1,0,0,0,0,1,0
2022-12-28,0,0,0,1.0,1.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,1.0,1,0,0,0,0,0,1
2022-12-29,1,1,0,2.0,2.0,2.0,0,0.0,1.0,1.0,...,0.0,1.0,1.0,1,0,0,0,1,0,0
2022-12-30,1,1,1,1.0,1.0,3.0,0,1.0,0.0,1.0,...,0.0,0.0,1.0,1,0,0,0,0,0,0
2022-12-31,1,0,1,1.0,1.0,1.0,0,1.0,1.0,0.0,...,1.0,0.0,0.0,1,0,1,0,0,0,0
2023-01-01,1,0,1,1.0,2.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,0.0,1,0,0,1,0,0,0
2023-01-02,1,1,0,1.0,3.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,1.0,1,1,0,0,0,0,0


## **2.  Constructing the Usage Agent**

### **2.1 Initialize Agent**

First we define the **Usage Agent class**. It takes as input the data generated by the prep.pipeline_usage function computed above, and the name of the device for which predictions should be made (e.g "Washing Machine", "Dishwasher"etc...).

In [6]:
class Usage_Agent:
    import pandas as pd

    def __init__(self, input_df, device):
        self.input = input_df
        self.device = device

Here we initialize the agent for the device "Dishwasher"

In [7]:
import pandas as pd
Usage_Agent_i = Usage_Agent(df, shiftable_devices[0]) 

### **2.2 Train_test_split function**

In [8]:
#train start: the day from which training starts
def get_train_start(self, df):
    import datetime
    end_date = min(df.index) + datetime.timedelta(days=3)
    # determine train_start date 
    return str(end_date)[:10]

# add to Activity agent
setattr(Usage_Agent, 'get_train_start', get_train_start)
del get_train_start 

In [9]:
Usage_Agent_i = Usage_Agent(df, shiftable_devices[0])
train_start = Usage_Agent_i.get_train_start(df)
train_start

'2022-12-28'

The number of data points available to make a prediction for day t increases by one, each time t increases by one. Therefore, we define a custom train_test_split function that automatically puts all data available until day t-1 (incl.) into the training set. The Data for day t (= prediction day) comes into the test set.

In order to limit over-fitting the function also filters out the number of features to be taken into account to train the model. Here these are the following:

1. Indicator of device usage at day t-1.
2. Indicator of device usage at day t-2.
3. Indicator of activity in the household in the past two days.


In [10]:
#date: the day of prediction
#train start: the day from which training starts
def train_test_split(self, df, date, train_start = ''):
    if train_start == '':
        train_start = self.get_train_start(df)
    
    #restrict number of variables
    select_vars =  [str(self.device) + '_usage', str(self.device)+ '_usage_lag_1', str(self.device)+ '_usage_lag_2',	'active_last_2_days']
    df = df[select_vars]
    #spli train and test
    X_train = df.loc[train_start:date, df.columns != str(self.device) + '_usage']
    y_train = df.loc[train_start:date, df.columns == str(self.device) + '_usage']
    X_test  = df.loc[date, df.columns != str(self.device) + '_usage']
    y_test  = df.loc[date , df.columns == str(self.device) + '_usage']
    return X_train, y_train, X_test, y_test

# add to Activity agent
setattr(Usage_Agent, 'train_test_split', train_test_split)
del train_test_split 

Ouput:

In [11]:
X_train, y_train, X_test, y_test = Usage_Agent_i.train_test_split(df, date, train_start)

In [12]:
X_train

,sensor.shellyplug_s_4022d88961b4_power_usage_lag_1,sensor.shellyplug_s_4022d88961b4_power_usage_lag_2,active_last_2_days
last_updated,,,
2022-12-28,1.0,1.0,1
2022-12-29,0.0,1.0,1
2022-12-30,1.0,0.0,1
2022-12-31,1.0,1.0,1
2023-01-01,0.0,1.0,1
2023-01-02,0.0,0.0,1
2023-01-03,1.0,0.0,1
2023-01-04,0.0,1.0,1
2023-01-05,0.0,0.0,0


In [13]:
X_test

sensor.shellyplug_s_4022d88961b4_power_usage_lag_1    0.0
sensor.shellyplug_s_4022d88961b4_power_usage_lag_2    0.0
active_last_2_days                                    1.0
Name: 2023-01-08 00:00:00, dtype: float64

In [14]:
y_train

,sensor.shellyplug_s_4022d88961b4_power_usage
last_updated,
2022-12-28,0
2022-12-29,1
2022-12-30,1
2022-12-31,0
2023-01-01,0
2023-01-02,1
2023-01-03,0
2023-01-04,0
2023-01-05,0


In [15]:
y_test

sensor.shellyplug_s_4022d88961b4_power_usage    1.0
Name: 2023-01-08 00:00:00, dtype: float64

### **2.3 Fitting Models**

Now that we have the function to perform the split-sampling we can fit the model on training data. For that purpose, we define a Logit-fitting function as follows:

In [16]:
def fit_Logit(self, X, y, max_iter=100):
    return LogisticRegression(random_state=0, max_iter=max_iter).fit(X, y)

def fit_knn(self, X, y, n_neighbors=10, leaf_size=30):
    return KNeighborsClassifier(n_neighbors=n_neighbors, leaf_size=leaf_size, algorithm="auto", n_jobs=-1).fit(X, y)

def fit_random_forest(self, X, y, max_depth=10, n_estimators=500, max_features="sqrt"):
    return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)

def fit_ADA(self, X, y, learning_rate=0.1, n_estimators=100):
    return AdaBoostClassifier(learning_rate=learning_rate, n_estimators=n_estimators).fit(X, y)

def fit_XGB(self, X, y, learning_rate=0.1, max_depth=6, reg_lambda=1, reg_alpha=0):
    return xgboost.XGBClassifier(verbosity=0, use_label_encoder=False, learning_rate=learning_rate, max_depth=max_depth, reg_lambda=reg_lambda, reg_alpha=reg_alpha).fit(X, y)

def fit_EBM(self, X, y): 
    return ExplainableBoostingClassifier().fit(X,y)

def fit_smLogit(self, X, y):
    return statsmodels.api.Logit(y, X).fit(disp=False)

# add to Activity agent
setattr(Usage_Agent, 'fit_Logit', fit_Logit)
del fit_Logit 
setattr(Usage_Agent, 'fit_knn', fit_knn)
del fit_knn 
setattr(Usage_Agent, 'fit_random_forest', fit_random_forest)
del fit_random_forest 
setattr(Usage_Agent, 'fit_ADA', fit_ADA)
del fit_ADA 
setattr(Usage_Agent, 'fit_XGB', fit_XGB)
del fit_XGB 
setattr(Usage_Agent, 'fit_EBM', fit_EBM)
del fit_EBM 
setattr(Usage_Agent, 'fit_smLogit', fit_smLogit)
del fit_smLogit 

In [17]:
def fit(self, X, y, model_type, **args):
    model = None
    if model_type == "logit":
        model = self.fit_Logit(X, y, **args)
    elif model_type == "ada":
        model = self.fit_ADA(X, y, **args)
    elif model_type == "knn":
        model = self.fit_knn(X, y, **args)
    elif model_type == "random forest":
        model = self.fit_random_forest(X,y, **args)
    elif model_type == "xgboost":
        model = self.fit_XGB(X,y, **args)
    elif model_type == "ebm":
        model = self.fit_EBM(X,y, **args)
    elif model_type == "logit_sm":
        model = self.fit_smLogit(X, y)
    else:
        raise InputError("Unknown model type.")
    return model
# add to Usage agent
setattr(Usage_Agent, 'fit', fit)
del fit

Using this function on the training split, we can train our first model:

In [18]:
usage = Usage_Agent(df, shiftable_devices[0])
model = usage.fit(X_train, y_train, model_type)
model

C:\Users\chris\AppData\Local\Temp\ipykernel_13508\3690306017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)


RandomForestClassifier(max_depth=10, n_estimators=500, n_jobs=-1)

In [19]:
y_train

,sensor.shellyplug_s_4022d88961b4_power_usage
last_updated,
2022-12-28,0
2022-12-29,1
2022-12-30,1
2022-12-31,0
2023-01-01,0
2023-01-02,1
2023-01-03,0
2023-01-04,0
2023-01-05,0


Once the model is fitted to the training data, a prediction can be made for the test day. This prediction function is defined in the following:

In [20]:
def predict(self, model, X):
    import numpy as np
    import pandas
    res = 3
    cols = ["temp", "dwpt", "rhum", "wdir", "wspd"]
    for e in cols:
        if isinstance(X, pd.DataFrame):
            if e in X.columns:
                res += 1
        if isinstance(X, pd.Series):
            if e in X.index:
                res += 1
    X = np.array(X).reshape(-1, res)
    if type(model) == sklearn.linear_model.LogisticRegression:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == sklearn.neighbors._classification.KNeighborsClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == sklearn.ensemble._forest.RandomForestClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) ==  sklearn.ensemble._weight_boosting.AdaBoostClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == xgboost.sklearn.XGBClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == ExplainableBoostingClassifier:
        y_hat = model.predict_proba(X)[:,1]
    elif type(model) == statsmodels.discrete.discrete_model.BinaryResultsWrapper:
        y_hat = model.predict(X)
    else:
        raise InputError("Unknown model type.")

    return y_hat
# add to Usage agent
setattr(Usage_Agent, 'predict', predict)
del predict

In [21]:
#compute prediction at day t (see date used for split sampling)
import numpy as np
y_hat = usage.predict(model, X_test)
y_hat

C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0.65719048])

### **2.4 Pipeline**

Finally, we wrap up all the previously defined functions into the **pipeline** function. This allows to generate a prediction by simply inputting:
* the pre-processed usage data
* the prediction date
* the model type (limited to logit for now)
* the date at which the model has started to train


In [22]:
# pipeline function: predicting device usage
# -------------------------------------------------------------------------------------------
def pipeline(self, df, date, model_type, train_start='', weather_sel=False):

    if weather_sel:
        # Add Weather
        ################################
        from meteostat import Point, Daily
        from datetime import datetime, timedelta

        lough = Point(52.766593, -1.223511)
        time = df.index.to_series(name="time").tolist()
        start = time[0]
        end = time[len(time) - 1]
        weather = Daily(lough, start, end)
        weather = weather.fetch()

        from sklearn.impute import KNNImputer
        import numpy as np

        headers = weather.columns.values

        empty_train_columns = []
        for col in weather.columns.values:
            if sum(weather[col].isnull()) == weather.shape[0]:
                empty_train_columns.append(col)
        headers = np.setdiff1d(headers, empty_train_columns)

        imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
        weather = imputer.fit_transform(weather)
        scaler = MinMaxScaler()
        weather = scaler.fit_transform(weather)
        weather = pd.DataFrame(weather)
        weather["time"] = time[0:len(weather)]
        df["time"] = time

        weather.columns = np.append(headers, "time")

        df = pd.merge(df, weather, how="right", on="time")
        df = df.set_index("time")
        ################################

    X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)
    model = self.fit(X_train, y_train, model_type)
    return self.predict(model, X_test)

# add to Usage_Agent
setattr(Usage_Agent, 'pipeline', pipeline)
del pipeline


In [23]:
# pipeline function: predicting device usage
# -------------------------------------------------------------------------------------------
def pipeline_xai(self, df, date, model_type, train_start='', weather_sel=False):

    if weather_sel:
        # Add Weather
        ################################
        from meteostat import Point, Daily
        from datetime import datetime, timedelta

        lough = Point(52.766593, -1.223511)
        time = df.index.to_series(name="time").tolist()
        start = time[0]
        end = time[len(time) - 1]
        weather = Daily(lough, start, end)
        weather = weather.fetch()

        from sklearn.impute import KNNImputer
        import numpy as np

        headers = weather.columns.values

        empty_train_columns = []
        for col in weather.columns.values:
            if sum(weather[col].isnull()) == weather.shape[0]:
                empty_train_columns.append(col)
        headers = np.setdiff1d(headers, empty_train_columns)

        imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
        weather = imputer.fit_transform(weather)
        scaler = MinMaxScaler()
        weather = scaler.fit_transform(weather)
        weather = pd.DataFrame(weather)
        weather["time"] = time[0:len(weather)]
        df["time"] = time

        weather.columns = np.append(headers, "time")

        df = pd.merge(df, weather, how="right", on="time")
        df = df.set_index("time")
        ################################

    X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)
    model = self.fit(X_train, y_train, model_type)
    return self.predict(model, X_test), X_train, X_test, model

# add to Usage_Agent
setattr(Usage_Agent, 'pipeline_xai', pipeline_xai)
del pipeline_xai


A prediction for the "2013-12-08" based on the data starting on the '2013-11-01' can finally be made for the device with which we initialized the class (here: "Dishwasher")

In [24]:
usage.pipeline(df, date, 'logit')

C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0.41667434])

In [27]:
usage.pipeline_xai(df, date, 'logit', train_start = '2022-12-29')

C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


(array([0.42481799]),
               sensor.shellyplug_s_4022d88961b4_power_usage_lag_1  \
 last_updated                                                       
 2022-12-29                                                  0.0    
 2022-12-30                                                  1.0    
 2022-12-31                                                  1.0    
 2023-01-01                                                  0.0    
 2023-01-02                                                  0.0    
 2023-01-03                                                  1.0    
 2023-01-04                                                  0.0    
 2023-01-05                                                  0.0    
 2023-01-06                                                  0.0    
 2023-01-07                                                  0.0    
 2023-01-08                                                  0.0    
 
               sensor.shellyplug_s_4022d88961b4_power_usage_lag_2  \
 last_upda

### **2.5 Model Evaluation**

Finally, we want to assess the accuracy of our model before using it in the Recommendation Agent. 

A drawback to our approach is that we are not able to apply conventional model evaluation techniques to our model. We will train our model for each day to account for newly available information. Hence, we have different train and test sets for each day and for each day different performance metric based on the respective data sets. Therefore, we created our own evaluation function. 

Our evaluation function will build a model, fit the model and predict the target for each day for a given prediction period. For each day and fitted model it will calculate a performance metric on the train data. We chose the Area Under the Receiver Operating Characteristic Curve (AUC) as performance metric for our binary classification task. As in our case the test data is only the current date to be predicted, we calculate the test AUC over the usage probabilities of all day after all days have been predicted. To summarize the train AUC in one score, we apply an average over all calculated train AUC scores (Note: This approach is the same as for the activity predictions).

In [28]:
def auc(self, y_true, y_hat):
    import sklearn.metrics
    return sklearn.metrics.roc_auc_score(y_true, y_hat)
# add to Usage agent
setattr(Usage_Agent, 'auc', auc)
del auc

In [29]:
def evaluate(
        self, df, model_type, train_start='', predict_start="2014-01-01", predict_end=-1, return_errors=False,
        weather_sel=False, xai=False, **args
):
    import pandas as pd
    import numpy as np
    from tqdm import tqdm

    dates = pd.DataFrame(df.index)
    dates = dates.set_index(df.index)["last_updated"]
    predict_start = pd.to_datetime(predict_start)
    predict_end = (
        pd.to_datetime(dates.iloc[predict_end])
        if type(predict_end) == int
        else pd.to_datetime(predict_end)
    )
    dates = dates.loc[predict_start:predict_end]
    y_true = []
    y_hat_train = {}
    y_hat_test = []
    y_hat_lime = []
    y_hat_shap = []
    auc_train_dict = {}
    auc_test = []
    xai_time_lime = []
    xai_time_shap = []

    predictions_list = []

    if weather_sel:
        print('Crawl weather data....')
        # Add Weather
        ################################
        from meteostat import Point, Daily
        from datetime import datetime, timedelta

        lough = Point(52.766593, -1.223511)
        time = df.index.to_series(name="time").tolist()
        start = time[0]
        end = time[len(time) - 1]
        weather = Daily(lough, start, end)
        weather = weather.fetch()

        from sklearn.impute import KNNImputer
        import numpy as np

        headers = weather.columns.values

        empty_train_columns = []
        for col in weather.columns.values:
            if sum(weather[col].isnull()) == weather.shape[0]:
                empty_train_columns.append(col)
        headers = np.setdiff1d(headers, empty_train_columns)

        imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
        weather = imputer.fit_transform(weather)
        scaler = MinMaxScaler()
        weather = scaler.fit_transform(weather)
        weather = pd.DataFrame(weather)
        weather["time"] = time[0:len(weather)]
        df["time"] = time

        weather.columns = np.append(headers, "time")

        df = pd.merge(df, weather, how="right", on="time")
        df = df.set_index("time")

        ################################

    if not xai:
        for date in tqdm(dates.index):
            errors = {}
            try:
                X_train, y_train, X_test, y_test = self.train_test_split(
                    df, date, train_start
                )
                # fit model
                model = self.fit(X_train, y_train, model_type, **args)
                # predict
                y_hat_train.update({date: self.predict(model, X_train)})
                y_hat_test += list(self.predict(model, X_test))
                # evaluate train data
                auc_train_dict.update(
                    {date: self.auc(y_train, list(y_hat_train.values())[-1])}
                )
                y_true += list(y_test)

            except Exception as e:
                errors[date] = e
    else:
        print('The explainability approaches in the Usage Agent are being evaluated for model: ' + str(model_type))
        print('Start evaluation with LIME and SHAP')
        import time
        import lime
        import shap as shap
        from lime import lime_tabular

        for date in tqdm(dates.index):
            errors = {}
            try:
                X_train, y_train, X_test, y_test = self.train_test_split(
                    df, date, train_start
                )
                # fit model
                model = self.fit(X_train, y_train, model_type)
                # predict
                y_hat_train.update({date: self.predict(model, X_train)})
                y_hat_test += list(self.predict(model, X_test))
                # evaluate train data
                auc_train_dict.update(
                    {date: self.auc(y_train, list(y_hat_train.values())[-1])}
                )
                y_true += list(y_test)
                start_time = time.time()

                if model_type == "xgboost":
                    booster = model.get_booster()

                    explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,
                                                                       feature_names=X_train.columns,
                                                                       kernel_width=3, verbose=False)

                else:
                    explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train),
                                                                  mode="classification",
                                                                  feature_names=X_train.columns,
                                                                  categorical_features=[0])

                if model_type == "xgboost":
                    exp = explainer.explain_instance(X_test, model.predict_proba)
                else:
                    exp = explainer.explain_instance(data_row=X_test, predict_fn=model.predict_proba)

                y_hat_lime += list(exp.local_pred)

                # take time for each day:
                end_time = time.time()
                difference_time = end_time - start_time

                xai_time_lime.append(difference_time)
                # SHAP
                # =========================================================================
                start_time = time.time()

                if model_type == "logit":
                    X_train_summary = shap.sample(X_train, 100)
                    explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)


                elif model_type == "ada":
                    X_train_summary = shap.sample(X_train, 100)
                    explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

                elif model_type == "knn":
                    X_train_summary = shap.sample(X_train, 100)
                    explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)


                elif model_type == "random forest":
                    X_train_summary = shap.sample(X_train, 100)
                    explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

                elif model_type == "xgboost":
                    explainer = shap.TreeExplainer(model, X_train, model_output='predict_proba')

                else:
                    raise InputError("Unknown model type.")

                base_value = explainer.expected_value[1]  # the mean prediction


                shap_values = explainer.shap_values(
                    X_test)
                contribution_to_class_1 = np.array(shap_values).sum(axis=1)[1]  # the red part of the diagram
                shap_prediction = base_value + contribution_to_class_1
                # Prediction from XAI:
                y_hat_shap += list([shap_prediction])


                # take time for each day:
                end_time = time.time()
                difference_time = end_time - start_time
                xai_time_shap.append(difference_time)

            except Exception as e:
                errors[date] = e
    
    auc_test = self.auc(y_true, y_hat_test)
    auc_train = np.mean(list(auc_train_dict.values()))
    predictions_list.append(y_true)
    predictions_list.append(y_hat_test)
    predictions_list.append(y_hat_lime)
    predictions_list.append(y_hat_shap)

    # Efficiency
    time_mean_lime = np.mean(xai_time_lime)
    time_mean_shap = np.mean(xai_time_shap)
    print('Mean time needed by appraoches: ' + str(time_mean_lime) + ' ' + str(time_mean_shap))

    if return_errors:
        return auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list, errors
    else:
        return auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list
# add to Usage agent
setattr(Usage_Agent, 'evaluate', evaluate)
del evaluate

In [30]:
predict_end=-1
return_errors=False
weather_sel=False
xai=True
model_type = "random forest"
predict_start='2022-12-27'

Finally, we can evaluate the simple Logit model for the "Dishwasher", for instance for all predictions after the "2014-08-01".  

In [31]:
auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list, = usage.evaluate(df, model_type = model_type, predict_start=predict_start, predict_end= -1)
print("mean_auc_on_train = "+ str(auc_train) + " | test_auc = " + str(auc_test))

  0%|                                                                                           | 0/14 [00:00<?, ?it/s]C:\Users\chris\AppData\Local\Temp\ipykernel_13508\3690306017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
 14%|███████████▊                                                                       | 2/14 [00:00<00:04,  2.92it/s]C:\Users\chris\AppData\Local\Temp\ipykernel_13508\3690306017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using

 79%|████████████████████████████████████████████████████████████████▍                 | 11/14 [00:08<00:02,  1.21it/s]C:\Users\chris\AppData\Local\Temp\ipykernel_13508\3690306017.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
 86%|██████████████████████████████████████████████████████████████████████▎           | 12/14 [00:09<00:01,  1.20it/s]C:\Users\chr

Mean time needed by appraoches: nan nan
mean_auc_on_train = 0.8912657076719577 | test_auc = 1.0



C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [32]:
predictions_list

[[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.682,
  0.74,
  0.07,
  0.4480333333333333,
  0.8893666666666664,
  0.4869666666666669,
  0.3407857142857144,
  0.3046666666666667,
  0.108,
  0.46620000000000006,
  0.6358730158730163,
  0.33423809523809545],
 [],
 []]

As can be seen above, the model's performance is quite disappointing. It is not surprising that we do not have a very high accuracy, given the little amount of data we have. However, there must be potential for improvment. A first step in that direction would be a proper feature selection methodology taking into account different devices and households. Moreover, there has been a large decrease in model accuracy after changing the pre-processing pipeline methodology. Therefore, it seems that the model is sensitive to the way we detect the devices' activity. In the next steps we should investigate how and why these pre-processing steps impact the model's performance.

## **Appendix A1: Complete Usage Agent Class**

In [33]:
class Usage_Agent:
    import pandas as pd

    def __init__(self, input_df, device):
        self.input = input_df
        self.device = device

    # train test split
    # -------------------------------------------------------------------------------------------
    
    #train start: the day from which training starts
    def get_train_start(self, df):
        end_date = min(df.index) + datetime.timedelta(days=3)
        # determine train_start date 
        return str(end_date)[:10]
    
    def train_test_split(self, df, date, train_start=''):
        
        select_vars =  [str(self.device) + '_usage', 
                        str(self.device)+ '_usage_lag_1', 
                        str(self.device)+ '_usage_lag_2', 'active_last_2_days']
        
        if train_start == '':
            train_start = self.get_train_start(df)
            
        # Add weather possibly
        if "temp" in df.columns:
            select_vars.append("temp")
            df["temp"].fillna(method="backfill", inplace=True)
        if "dwpt" in df.columns:
            select_vars.append("dwpt")
            df["dwpt"].fillna(method="backfill", inplace=True)
        if "rhum" in df.columns:
            select_vars.append("rhum")
            df["rhum"].fillna(method="backfill", inplace=True)
        if "wdir" in df.columns:
            select_vars.append("wdir")
            df["wdir"].fillna(method="backfill", inplace=True)
        if "wspd" in df.columns:
            select_vars.append("wspd")
            df["wspd"].fillna(method="backfill", inplace=True)
        
        df = df[select_vars]
        X_train = df.loc[train_start:date, df.columns != str(self.device) + "_usage"]
        y_train = df.loc[train_start:date, df.columns == str(self.device) + "_usage"]
        X_test = df.loc[date, df.columns != str(self.device) + "_usage"]
        y_test = df.loc[date, df.columns == str(self.device) + "_usage"]
        return X_train, y_train, X_test, y_test
    
    # model training and evaluation
    # -------------------------------------------------------------------------------------------
    def fit_Logit(self, X, y, max_iter=100):
        from sklearn.linear_model import LogisticRegression
        return LogisticRegression(random_state=0, max_iter=max_iter).fit(X, y)

    def fit_knn(self, X, y, n_neighbors=10, leaf_size=30):
        from sklearn.neighbors import KNeighborsClassifier
        return KNeighborsClassifier(n_neighbors=n_neighbors, leaf_size=leaf_size, algorithm="auto", n_jobs=-1).fit(X, y)

    def fit_random_forest(self, X, y, max_depth=10, n_estimators=500, max_features="sqrt"):
        from sklearn.ensemble import RandomForestClassifier
        return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)

    def fit_ADA(self, X, y, learning_rate=0.1, n_estimators=100):
        from sklearn.ensemble import AdaBoostClassifier
        return AdaBoostClassifier(learning_rate=learning_rate, n_estimators=n_estimators).fit(X, y)

    def fit_XGB(self, X, y, learning_rate=0.1, max_depth=6, reg_lambda=1, reg_alpha=0):
        import xgboost
        return xgboost.XGBClassifier(verbosity=0, use_label_encoder=False, learning_rate=learning_rate, max_depth=max_depth, reg_lambda=reg_lambda, reg_alpha=reg_alpha).fit(X, y)

    def fit_EBM(self, X, y): 
        from interpret.glassbox.ebm.ebm import ExplainableBoostingClassifier  
        return ExplainableBoostingClassifier().fit(X,y)

    def fit_smLogit(self, X, y):
        import statsmodels
        return statsmodels.api.Logit(y, X).fit(disp=False)
    
    def fit(self, X, y, model_type, **args):
        model = None
        if model_type == "logit":
            model = self.fit_Logit(X, y, **args)
        elif model_type == "ada":
            model = self.fit_ADA(X, y, **args)
        elif model_type == "knn":
            model = self.fit_knn(X, y, **args)
        elif model_type == "random forest":
            model = self.fit_random_forest(X, y, **args)
        elif model_type == "xgboost":
            model = self.fit_XGB(X, y, **args)
        elif model_type == "ebm":
            model = self.fit_EBM(X,y, **args)
        elif model_type == "logit_sm":
            model = self.fit_smLogit(X, y)
        else:
            raise InputError("Unknown model type.")
        return model

    def predict(self, model, X):
        import sklearn
        import statsmodels
        from interpret.glassbox.ebm.ebm import ExplainableBoostingClassifier     
        from sklearn.linear_model import LogisticRegression
        from sklearn.neighbors import KNeighborsClassifier
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.ensemble import AdaBoostClassifier
        import xgboost
        import numpy as np
        import pandas
        res = 3
        cols = ["temp", "dwpt", "rhum", "wdir", "wspd"]
        for e in cols:
            if isinstance(X, pd.DataFrame):
                if e in X.columns:
                    res += 1
            if isinstance(X, pd.Series):
                if e in X.index:
                    res += 1
        X = np.array(X).reshape(-1, res)
        if type(model) == sklearn.linear_model.LogisticRegression:
            y_hat = model.predict_proba(X)[:,1]
        elif type(model) == sklearn.neighbors._classification.KNeighborsClassifier:
            y_hat = model.predict_proba(X)[:,1]
        elif type(model) == sklearn.ensemble._forest.RandomForestClassifier:
            y_hat = model.predict_proba(X)[:,1]
        elif type(model) ==  sklearn.ensemble._weight_boosting.AdaBoostClassifier:
            y_hat = model.predict_proba(X)[:,1]
        elif type(model) == xgboost.sklearn.XGBClassifier:
            y_hat = model.predict_proba(X)[:,1]
        elif type(model) == ExplainableBoostingClassifier:
            y_hat = model.predict_proba(X)[:,1]
        elif type(model) == statsmodels.discrete.discrete_model.BinaryResultsWrapper:
            y_hat = model.predict(X)
        else:
            raise InputError("Unknown model type.")
        return y_hat

    def auc(self, y_true, y_hat):
        import sklearn.metrics
        return sklearn.metrics.roc_auc_score(y_true, y_hat)
    
    def evaluate(
            self, df, model_type, train_start = '', predict_start="2014-01-01", predict_end=-1, return_errors=False,
            weather_sel=False, xai=False, **args
    ):
        import pandas as pd
        import numpy as np
        from tqdm import tqdm

        dates = pd.DataFrame(df.index)
        dates = dates.set_index(df.index)["last_updated"]
        predict_start = pd.to_datetime(predict_start)
        predict_end = (
            pd.to_datetime(dates.iloc[predict_end])
            if type(predict_end) == int
            else pd.to_datetime(predict_end)
        )
        dates = dates.loc[predict_start:predict_end]
        y_true = []
        y_hat_train = {}
        y_hat_test = []
        y_hat_lime = []
        y_hat_shap = []
        auc_train_dict = {}
        auc_test = []
        xai_time_lime = []
        xai_time_shap = []

        predictions_list = []

        if weather_sel:
            print('Crawl weather data....')
            # Add Weather
            ################################
            from meteostat import Point, Daily
            from datetime import datetime, timedelta

            lough = Point(52.766593, -1.223511)
            time = df.index.to_series(name="time").tolist()
            start = time[0]
            end = time[len(time) - 1]
            weather = Daily(lough, start, end)
            weather = weather.fetch()

            from sklearn.impute import KNNImputer
            import numpy as np

            headers = weather.columns.values

            empty_train_columns = []
            for col in weather.columns.values:
                if sum(weather[col].isnull()) == weather.shape[0]:
                    empty_train_columns.append(col)
            headers = np.setdiff1d(headers, empty_train_columns)

            imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
            weather = imputer.fit_transform(weather)
            scaler = MinMaxScaler()
            weather = scaler.fit_transform(weather)
            weather = pd.DataFrame(weather)
            weather["time"] = time[0:len(weather)]
            df["time"] = time

            weather.columns = np.append(headers, "time")

            df = pd.merge(df, weather, how="right", on="time")
            df = df.set_index("time")

            ################################

        if not xai:
            for date in tqdm(dates.index):
                errors = {}
                try:
                    X_train, y_train, X_test, y_test = self.train_test_split(
                        df, date, train_start
                    )
                    # fit model
                    model = self.fit(X_train, y_train, model_type, **args)
                    # predict
                    y_hat_train.update({date: self.predict(model, X_train)})
                    y_hat_test += list(self.predict(model, X_test))
                    # evaluate train data
                    auc_train_dict.update(
                        {date: self.auc(y_train, list(y_hat_train.values())[-1])}
                    )
                    y_true += list(y_test)

                except Exception as e:
                    errors[date] = e
        else:
            print('The explainability approaches in the Usage Agent are being evaluated for model: ' + str(model_type))
            print('Start evaluation with LIME and SHAP')
            import time
            import lime
            import shap as shap
            from lime import lime_tabular

            for date in tqdm(dates.index):
                errors = {}
                try:
                    X_train, y_train, X_test, y_test = self.train_test_split(
                        df, date, train_start
                    )
                    # fit model
                    model = self.fit(X_train, y_train, model_type)
                    # predict
                    y_hat_train.update({date: self.predict(model, X_train)})
                    y_hat_test += list(self.predict(model, X_test))
                    # evaluate train data
                    auc_train_dict.update(
                        {date: self.auc(y_train, list(y_hat_train.values())[-1])}
                    )
                    y_true += list(y_test)
                    start_time = time.time()

                    if model_type == "xgboost":
                        booster = model.get_booster()

                        explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,
                                                                           feature_names=X_train.columns,
                                                                           kernel_width=3, verbose=False)

                    else:
                        explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train),
                                                                      mode="classification",
                                                                      feature_names=X_train.columns,
                                                                      categorical_features=[0])

                    if model_type == "xgboost":
                        exp = explainer.explain_instance(X_test, model.predict_proba)
                    else:
                        exp = explainer.explain_instance(data_row=X_test, predict_fn=model.predict_proba)

                    y_hat_lime += list(exp.local_pred)

                    # take time for each day:
                    end_time = time.time()
                    difference_time = end_time - start_time

                    xai_time_lime.append(difference_time)
                    # SHAP
                    # =========================================================================
                    start_time = time.time()

                    if model_type == "logit":
                        X_train_summary = shap.sample(X_train, 100)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)


                    elif model_type == "ada":
                        X_train_summary = shap.sample(X_train, 100)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

                    elif model_type == "knn":
                        X_train_summary = shap.sample(X_train, 100)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)


                    elif model_type == "random forest":
                        X_train_summary = shap.sample(X_train, 100)
                        explainer = shap.KernelExplainer(model.predict_proba, X_train_summary)

                    elif model_type == "xgboost":
                        explainer = shap.TreeExplainer(model, X_train, model_output='predict_proba')

                    else:
                        raise InputError("Unknown model type.")

                    base_value = explainer.expected_value[1]  # the mean prediction


                    shap_values = explainer.shap_values(
                        X_test)
                    contribution_to_class_1 = np.array(shap_values).sum(axis=1)[1]  # the red part of the diagram
                    shap_prediction = base_value + contribution_to_class_1
                    # Prediction from XAI:
                    y_hat_shap += list([shap_prediction])


                    # take time for each day:
                    end_time = time.time()
                    difference_time = end_time - start_time
                    xai_time_shap.append(difference_time)

                except Exception as e:
                    errors[date] = e

        auc_test = self.auc(y_true, y_hat_test)
        auc_train = np.mean(list(auc_train_dict.values()))
        predictions_list.append(y_true)
        predictions_list.append(y_hat_test)
        predictions_list.append(y_hat_lime)
        predictions_list.append(y_hat_shap)

        # Efficiency
        time_mean_lime = np.mean(xai_time_lime)
        time_mean_shap = np.mean(xai_time_shap)
        print('Mean time nedded by appraoches: ' + str(time_mean_lime) + ' ' + str(time_mean_shap))

        if return_errors:
            return auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list, errors
        else:
            return auc_train, auc_test, auc_train_dict, time_mean_lime, time_mean_shap, predictions_list
        
    # pipeline function: predicting device usage
    # -------------------------------------------------------------------------------------------
    def pipeline(self, df, date, model_type, train_start = '', weather_sel=False):

        if weather_sel:
            # Add Weather
            ################################
            from meteostat import Point, Daily
            from datetime import datetime, timedelta

            lough = Point(52.766593, -1.223511)
            time = df.index.to_series(name="time").tolist()
            start = time[0]
            end = time[len(time) - 1]
            weather = Daily(lough, start, end)
            weather = weather.fetch()

            from sklearn.impute import KNNImputer
            import numpy as np

            headers = weather.columns.values

            empty_train_columns = []
            for col in weather.columns.values:
                if sum(weather[col].isnull()) == weather.shape[0]:
                    empty_train_columns.append(col)
            headers = np.setdiff1d(headers, empty_train_columns)

            imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
            weather = imputer.fit_transform(weather)
            scaler = MinMaxScaler()
            weather = scaler.fit_transform(weather)
            weather = pd.DataFrame(weather)
            weather["time"] = time[0:len(weather)]
            df["time"] = time

            weather.columns = np.append(headers, "time")

            df = pd.merge(df, weather, how="right", on="time")
            df = df.set_index("time")
            ################################

        X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)
        model = self.fit(X_train, y_train, model_type)
        return self.predict(model, X_test)


    # pipeline function: predicting device usage
    # -------------------------------------------------------------------------------------------
    def pipeline_xai(self, df, date, model_type, train_start = '', weather_sel=False):

        if weather_sel:
            # Add Weather
            ################################
            from meteostat import Point, Daily
            from datetime import datetime, timedelta

            lough = Point(52.766593, -1.223511)
            time = df.index.to_series(name="time").tolist()
            start = time[0]
            end = time[len(time) - 1]
            weather = Daily(lough, start, end)
            weather = weather.fetch()

            from sklearn.impute import KNNImputer
            import numpy as np

            headers = weather.columns.values

            empty_train_columns = []
            for col in weather.columns.values:
                if sum(weather[col].isnull()) == weather.shape[0]:
                    empty_train_columns.append(col)
            headers = np.setdiff1d(headers, empty_train_columns)

            imputer = KNNImputer(missing_values=np.nan, n_neighbors=7, weights="distance")
            weather = imputer.fit_transform(weather)
            scaler = MinMaxScaler()
            weather = scaler.fit_transform(weather)
            weather = pd.DataFrame(weather)
            weather["time"] = time[0:len(weather)]
            df["time"] = time

            weather.columns = np.append(headers, "time")

            df = pd.merge(df, weather, how="right", on="time")
            df = df.set_index("time")
            ################################

        X_train, y_train, X_test, y_test = self.train_test_split(df, date, train_start)
        model = self.fit(X_train, y_train, model_type)
        return self.predict(model, X_test), X_train, X_test, model

In [34]:
usage_1 = Usage_Agent(df, shiftable_devices[0])
usage_2 = Usage_Agent(df, shiftable_devices[1])
date = '2023-01-08'
train_start = ''
output = usage_1.pipeline_xai(df, date, 'random forest', train_start)
output

C:\Users\chris\AppData\Local\Temp\ipykernel_13508\3757562704.py:62: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


(array([0.67094762]),
               sensor.shellyplug_s_4022d88961b4_power_usage_lag_1  \
 last_updated                                                       
 2022-12-28                                                  1.0    
 2022-12-29                                                  0.0    
 2022-12-30                                                  1.0    
 2022-12-31                                                  1.0    
 2023-01-01                                                  0.0    
 2023-01-02                                                  0.0    
 2023-01-03                                                  1.0    
 2023-01-04                                                  0.0    
 2023-01-05                                                  0.0    
 2023-01-06                                                  0.0    
 2023-01-07                                                  0.0    
 2023-01-08                                                  0.0    
 
          

In [35]:
output = usage_2.pipeline_xai(df, date, 'random forest', train_start)
output

C:\Users\chris\AppData\Local\Temp\ipykernel_13508\3757562704.py:62: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


(array([0.01449524]),
               sensor.shellyplug_s_4022d88984b8_power_usage_lag_1  \
 last_updated                                                       
 2022-12-28                                                  1.0    
 2022-12-29                                                  0.0    
 2022-12-30                                                  0.0    
 2022-12-31                                                  1.0    
 2023-01-01                                                  1.0    
 2023-01-02                                                  1.0    
 2023-01-03                                                  0.0    
 2023-01-04                                                  0.0    
 2023-01-05                                                  0.0    
 2023-01-06                                                  0.0    
 2023-01-07                                                  1.0    
 2023-01-08                                                  0.0    
 
          